Azure ML & Azure Databricks notebooks by Parashar Shah.

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/azure-databricks/amlsdk/deploy-to-aks-existingimage-05.png)

This notebook uses image from ACI notebook for deploying to AKS.

In [ ]:
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

In [ ]:
# Set auth to be used by workspace related APIs.
# For automation or CI/CD ServicePrincipalAuthentication can be used.
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.authentication.serviceprincipalauthentication?view=azure-ml-py
auth = None

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config(auth = auth)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
# List images by ws

from azureml.core.image import ContainerImage
for i in ContainerImage.list(workspace = ws):
    print('{}(v.{} [{}]) stored at {} with build log {}'.format(i.name, i.version, i.creation_state, i.image_location, i.image_build_log_uri))

In [ ]:
from azureml.core.image import Image
myimage = Image(workspace=ws, name="aciws")

In [ ]:
#create AKS compute
#it may take 20-25 minutes to create a new cluster

from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'ps-aks-demo2' 

# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

aks_target.wait_for_completion(show_output = True)

print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

In [ ]:
from azureml.core.webservice import Webservice
help( Webservice.deploy_from_image)

In [ ]:
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import ContainerImage

#Set the web service configuration (using default here with app insights)
aks_config = AksWebservice.deploy_configuration(enable_app_insights=True)

#unique service name
service_name ='ps-aks-service'

# Webservice creation using single command, there is a variant to use image directly as well.
aks_service = Webservice.deploy_from_image(
  workspace=ws, 
  name=service_name,
  deployment_config = aks_config,
  image = myimage,
  deployment_target = aks_target
    )

aks_service.wait_for_deployment(show_output=True)

In [ ]:
aks_service.deployment_status

In [ ]:
#for using the Web HTTP API 
print(aks_service.scoring_uri)
print(aks_service.get_keys())

In [ ]:
import json

#get the some sample data
test_data_path = "AdultCensusIncomeTest"
test = spark.read.parquet(test_data_path).limit(5)

test_json = json.dumps(test.toJSON().collect())

print(test_json)

In [ ]:
#using data defined above predict if income is >50K (1) or <=50K (0)
aks_service.run(input_data=test_json)

In [ ]:
#comment to not delete the web service
aks_service.delete()
#image.delete()
#model.delete()
aks_target.delete() 

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/azure-databricks/amlsdk/deploy-to-aks-existingimage-05.png)